# Combining income with other attributes

In [1]:
import cudf
import pyarrow as pa
import pandas as pd
import numpy as np
import cupy as cp
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
df = cudf.read_parquet('../data/all_states_lat_long_education_age_sex.parquet/*')

In [4]:
df = df.sort_values('GISJOIN')
df

,GISJOIN,x,y,sex,p_id,age,education
68256,1000100201001000,-9.627138e+06,3.825075e+06,0,3,42,9
68257,1000100201001000,-9.627112e+06,3.824731e+06,0,4,40,9
68258,1000100201001000,-9.626588e+06,3.824990e+06,0,5,44,9
68259,1000100201001000,-9.627267e+06,3.824974e+06,0,6,41,9
68260,1000100201001000,-9.627082e+06,3.825165e+06,0,7,44,9
...,...,...,...,...,...,...,...
308737496,56004509513003128,-1.159921e+07,5.441573e+06,1,308745304,35,12
308737497,56004509513003128,-1.159932e+07,5.441618e+06,1,308745305,35,12
308737498,56004509513003128,-1.159941e+07,5.441687e+06,1,308745306,38,12
308737499,56004509513003128,-1.159941e+07,5.441688e+06,1,308745307,35,12


In [5]:
df.dtypes

GISJOIN        int64
x            float64
y            float64
sex             int8
p_id           int32
age             int8
education       int8
dtype: object

In [6]:
df_income = cudf.read_parquet('./income_step_2.parquet/*')

In [7]:
df_income.income = df_income.income.astype('int8')
df_income = df_income.sort_values('GISJOIN').reset_index()
df_income.drop_column('index')

In [8]:
df_income.dtypes

GISJOIN    int64
income      int8
dtype: object

In [9]:
df_income

,GISJOIN,income
0,1000100201001,8
1,1000100201001,8
2,1000100201001,8
3,1000100201001,8
4,1000100201001,8
...,...,...
243227903,56004509513003,17
243227904,56004509513003,17
243227905,56004509513003,17
243227906,56004509513003,17


In [10]:
df_income_with_gis_mappings = cudf.read_parquet('./GISJOIN_to_GISJOIN_og_mapping.parquet/*')
df_income_with_gis_mappings.age = df_income_with_gis_mappings.age.astype('int8')
# df_income_with_gis_mappings.drop_column('sex')

In [11]:
df_income_with_gis_mappings = df_income_with_gis_mappings.sort_values('GISJOIN').reset_index()
df_income_with_gis_mappings.drop_column('index')

In [12]:
df_income_with_gis_mappings.drop_column('GISJOIN_og')

In [13]:
df_income_with_gis_mappings

,GISJOIN,p_id,sex,age
0,1000100201001,213,1,68
1,1000100201001,214,1,68
2,1000100201001,215,1,68
3,1000100201001,216,1,68
4,1000100201001,217,1,67
...,...,...,...,...
243227903,56004509513003,308745019,0,24
243227904,56004509513003,308745020,0,24
243227905,56004509513003,308745021,0,24
243227906,56004509513003,308745022,0,23


In [14]:
df_income_with_gis_mappings['GISJOIN_income'] = df_income.GISJOIN.values

In [15]:
df_income_with_gis_mappings.query('GISJOIN != GISJOIN_income')

,GISJOIN,p_id,sex,age,GISJOIN_income


In [16]:
df_income_with_gis_mappings['income'] = df_income.income.values

In [17]:
df_income_with_gis_mappings.drop_column('GISJOIN_income')
df_income_with_gis_mappings.drop_column('GISJOIN')

In [18]:
df_income_with_gis_mappings.income.value_counts()

11    17378889
10    17111869
0     17089571
19    15838544
18    13956644
13    13742215
12    13653719
16    13449768
8     11940117
4     11903240
1     11820363
2     11401700
15    10797781
6     10778364
17     9688910
14     9660248
3      8942087
5      8715655
7      8010508
9      7347716
Name: income, dtype: int32

In [19]:
del(df_income)

### merge sex and income with lat-long table

In [20]:
df_income_with_gis_mappings

,p_id,sex,age,income
0,213,1,68,8
1,214,1,68,8
2,215,1,68,8
3,216,1,68,8
4,217,1,67,8
...,...,...,...,...
243227903,308745019,0,24,17
243227904,308745020,0,24,17
243227905,308745021,0,24,17
243227906,308745022,0,23,17


In [21]:
df = df.merge(df_income_with_gis_mappings, on='p_id', how='left')

In [22]:
df

,GISJOIN,x,y,sex_x,p_id,age_x,education,sex_y,age_y,income
0,1000100203001009,-9.624182e+06,3.826258e+06,0,5088,13,16,null,null,null
1,1000100203001009,-9.624108e+06,3.826253e+06,0,5089,12,16,null,null,null
2,1000100203001009,-9.624094e+06,3.826276e+06,0,5090,13,16,null,null,null
3,1000100203001009,-9.624169e+06,3.826045e+06,0,5091,10,16,null,null,null
4,1000100203001009,-9.624141e+06,3.826127e+06,0,5092,12,16,null,null,null
...,...,...,...,...,...,...,...,...,...,...
308745304,56004109752003864,-1.227103e+07,5.061271e+06,0,308712265,10,16,null,null,null
308745305,56004109752003864,-1.227072e+07,5.062986e+06,0,308712266,11,16,null,null,null
308745306,56004109752003864,-1.227213e+07,5.063594e+06,0,308712267,11,16,null,null,null
308745307,56004109752003864,-1.227194e+07,5.061883e+06,0,308712268,11,16,null,null,null


In [23]:
df.drop_column('age_y')
df.drop_column('sex_y')

In [24]:
# df.income.value_counts().sum()

In [25]:
df.income = df.income.fillna(20)
df.rename({
    'sex_x':'sex',
    'age_x': 'age'
}, inplace=True)

In [26]:
del(df_income_with_gis_mappings)

In [27]:
df

,GISJOIN,x,y,sex,p_id,age,education,income
0,1000100203001009,-9.624182e+06,3.826258e+06,0,5088,13,16,20
1,1000100203001009,-9.624108e+06,3.826253e+06,0,5089,12,16,20
2,1000100203001009,-9.624094e+06,3.826276e+06,0,5090,13,16,20
3,1000100203001009,-9.624169e+06,3.826045e+06,0,5091,10,16,20
4,1000100203001009,-9.624141e+06,3.826127e+06,0,5092,12,16,20
...,...,...,...,...,...,...,...,...
308745304,56004109752003864,-1.227103e+07,5.061271e+06,0,308712265,10,16,20
308745305,56004109752003864,-1.227072e+07,5.062986e+06,0,308712266,11,16,20
308745306,56004109752003864,-1.227213e+07,5.063594e+06,0,308712267,11,16,20
308745307,56004109752003864,-1.227194e+07,5.061883e+06,0,308712268,11,16,20


In [28]:
df.to_parquet('../data/all_states_lat_long_income_education_age_sex.parquet')

/home/ajay/anaconda3/envs/rapids_dev_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "
